In [1]:
import json
import re

In [2]:
with open("./rosmsgs.json", "r") as jsonFile:
    messages_data = json.load(jsonFile)
messages = sorted(messages_data.keys())

In [3]:
from msginfo import *

In [4]:
def collect_modules(messages):
    modules = {}
    for msg in messages:
        mi = MsgInfo(msg, messages_data[msg])
        if mi.module not in modules:
            modules[mi.module] = {}
        msgdata = { 
            'info'    : mi, 
            'depends' : mi.get_local_dependencies()
        }    
        modules[mi.module][mi.name] = msgdata  
    return modules

modules = collect_modules(messages)

In [5]:
# Deduplicating identifiers  
module = modules['Geometry_msgs'].copy()

for mod_name in modules:
    module = modules[mod_name]
    # Gather duplicate field names
    fields = {}
    for msg in module:
        for f in module[msg]['info'].get_field_names():
            if f not in fields:
                fields[f] = []
            fields[f].append(msg)

    duplicates = {}
    for field, msgs in fields.iteritems():
        if len(msgs) < 2: 
            continue
        for msg in msgs:
            if msg not in duplicates:
                duplicates[msg] = []
            duplicates[msg].append(field)

    # Apply prefix for duplicates
    for msg, fields in duplicates.iteritems():
        module[msg]['info'].prefix_fields(fields)


In [13]:
def write_types(mod_name, code):
    folder = "../imandra_model/src-messages"
    filename = folder + "/" + mod_name.lower() + ".ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Basic_types;;\n\n")
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")
            
def write_readers(mod_name, code):
    folder = "../imandra_model/src-messages-pp"
    filename = folder + "/" + mod_name.lower() + "_of_json.ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Json_utils;;\n")
        mlfile.write("open Basic_types_of_json;;\n")        
        mlfile.write("open Ros_messages.{};;\n\n".format(mod_name))        
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")   

In [16]:
folder = "../imandra_model/src-messages"
for mod_name in modules:
    module = modules[mod_name].copy()
    generated = set()
    types_code, readers_code = [], []
    while module:
        for m in module:
            deps = module[m]['depends']
            deps = set(deps) - generated 
            if not deps:
                generated.add(m)
                types_code.append(module[m]['info'].gen_typedecl())
                readers_code.append(module[m]['info'].gen_json_reader())
        for m in generated:
            if m in module:
                del module[m]
    
    write_types(mod_name, types_code)
    write_readers(mod_name, readers_code)
